In [37]:
import pandas as pd

# Binary or 3 label prediction?

Choose the label correctly


In [38]:
if_binary = 0

Compile the dataset now

In [39]:
df = pd.read_csv(
    "/Users/akseljoonas/Documents/news-sentiment/data/processed/news+prices-new-2.csv"
)

In [40]:
df['ticker'].value_counts()


ticker
ALVO     83
SCLX     66
QGEN     61
MDAI     60
VALN     59
         ..
MDXH     16
MYCOF    12
XBIT     10
ELOX      4
ADTX      1
Name: count, Length: 118, dtype: int64

In [41]:
# 1. Merge the text in 'title', 'summary', and 'description' into a new column 'text'
df["text"] = (
    df["title"].fillna("")
    + " "
    + df["summary"].fillna("")
)


In [42]:
# 2. Remove '<>:/\|?*' characters from the 'text' column
df["text"] = df["text"].str.replace(r"[<>:/\\|?*]", "", regex=True)


In [43]:
df.drop(columns=["labels"], inplace=True)
df.rename(columns={"5_labels": "labels"}, inplace=True)

In [44]:
df = df[["text", 'topic' ,"labels"]]

In [49]:
print(len(df))
df.drop_duplicates(inplace=True)
print(len(df))

4086
4069


In [51]:
df_relevant = df[df["labels"].isin([0, 1,3,4])]
df_irrelevant = df[df["labels"].isin([2])]
result = (
    df_relevant.groupby("topic")["labels"]
    .count()
    .reset_index(name="relevant_label_count")
)
result1 = (
    df_irrelevant.groupby("topic")["labels"]
    .count()
    .reset_index(name="irrelevant_label_count")
)
result = pd.merge(result, result1, on='topic', how="outer")

result_sorted = result.sort_values(by="relevant_label_count", ascending=False)
result_sorted

,topic,relevant_label_count,irrelevant_label_count
6,clinical_study,255.0,362.0
14,earnings_releases_and_operating_results,182.0,234.0
9,conference_calls__webcasts,164.0,345.0
3,calendar_of_events,105.0,293.0
25,health,105.0,185.0
42,product___services_announcement,99.0,210.0
22,financing_agreements,60.0,54.0
17,european_regulatory_news,56.0,223.0
41,press_releases,55.0,101.0
4,changes_in_companys_own_shares,42.0,68.0


In [52]:
min_label_count = 3

excluded_topics = [
    # "calendar_of_events",
    # "conference_calls__webcasts",
    # "directors_and_officers",
    # "company_announcements",
    # "european_regulatory_news",
]


# Filter topics with relevant_label_count >= 7
filtered_topics = result_sorted[
    result_sorted["relevant_label_count"] >= min_label_count
]["topic"].tolist()

# Remove excluded topics from the filtered topics
filtered_topics = [topic for topic in filtered_topics if topic not in excluded_topics]

# Now filter the original DataFrame based on the remaining filtered topics
df = df[df["topic"].isin(filtered_topics)]

# df.drop('topic',inplace=True, axis=1)
# Optionally, reset the index for cleanliness
df.reset_index(drop=True, inplace=True)

In [53]:
df['labels'].value_counts()

labels
2.0    2586
1.0     404
3.0     401
4.0     329
0.0     300
Name: count, dtype: int64

In [54]:
df.drop('topic',inplace=True, axis=1)

In [55]:
# 4. Save the modified DataFrame to a new CSV file
df.to_csv(
    "finetuning_5_labels_topic_pruned.csv",
    index=False,
)
